# PHM Society 2016

# Big Data in PHM

## Analysis of the 2008 PHM Data Set Using Python with Spark

### Presented by John Patanian, Principal Engineer GE Power
### October 2, 2016

## Abstract
The purpose of this Jupyter notebook is to demonstrate a typical anomaly detection of an using industrial time series data using the standard scientific Python tools stack. This will serve as a baseline comparison to the Apache Spark version, which can scale to a much larger than memory data set.
The dataset comes from the 2008 PHM Data Challenge, and consists a dataset of multiple parameters from a jet engine. The dataset includes data from several jet engines of a similar design and includes data for three different settings, as well as about 21 sensor readings.

For more information on the dataset, please refer to PHM 2008 Data Challenge Data Set Documentation.
In this notebook, we will do pre-processing of the data using the scientific Python stack, which includes pandas, numpy and matplotlib. This will provide exposure to several data preprocessing an analysis techniques that are commonly applies to analyses in the context of a PHM application.

## Prerequisites

This notebook assumes you already have a local version of Spark installed on your computer. In order for the Jupyter Notebook to be ** *Spark Aware* **, there are a few environment and path variables that need to be set. This can either be done at the top of the Notebook, in a .bashrc or .bash_profile file (OS-X or Linux), or in a shell script or batch file.

For reference purposes, below is a copy of my .bash_profile file on OS-X.

``` bash
export SPARK_HOME=/usr/local/spark
export PYLIB=/usr/local/spark/python/lib
export PATH=$PATH:$SPARK_HOME
export PATH=/Users/johnpatanian/anaconda/bin:$PATH   #Not added for Spark, but added by the Anaconda Installer
```

For reference, I used the following links to figure out how to do these settings:

[Instructions from Continuum](https://www.continuum.io/blog/developer-blog/using-anaconda-pyspark-distributed-language-processing-hadoop-cluster)

[Instructions from Cloudera](http://www.cloudera.com/documentation/enterprise/5-5-x/topics/spark_ipython.html#ipython__notebook_quickstart)

Note that the paths to your versions of these files depending on your installation details and operating system.

## Step 1: Set the Required environment and Path Variables

In [1]:
import os
import sys

# Note SPARK_HOME is set in my .bashrc file
#Set PYSPARK_PYTHON to the path of the local interpreter or environment
os.environ["PYSPARK_PYTHON"]="/Users/johnpatanian/anaconda/bin/python" 
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

## Step 2: Import libraries and create connection to the Spark Engine

The entry point to the spark engine is the Spark Context (sc). The SQLContext provides access to the Spark DataFrame as well as gives easy access to any data either in Hive tables, or any database accessible with jdbc (specific jdbc drivers may be required, if they are not installed by default).


In [2]:
import re

import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, functions as F
from pyspark.sql.functions import array
from pyspark.mllib.stat import Statistics
from pyspark.mllib.clustering import KMeans, KMeansModel

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('phm_demo')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

## Step 3: Connect/Import the Data

For this example, since the data set is not large, we are just going to use pandas to read in the CSV file and create a Spark DataFrame from the pandas DataFrame. For a *big* data set, we would use other methods to connect to a Hive table, data stored in HDFS, or data stored in other databases. Examples for each of these alternatives are beyond the scope of this tutorial, but once the data is in a DataFrame, all operations are identical. The source of the data is then abstracted from the user.



In [17]:
#Tag Names provided in data set description
tag_names = ['unit','cycle_num', 'setting1','setting2',
         'setting3', 'sensor1', 'sensor2',
         'sensor3', 'sensor4', 'sensor5', 'sensor6',
         'sensor7', 'sensor8', 'sensor9', 'sensor10',
         'sensor11', 'sensor12', 'sensor13', 'sensor14',
         'sensor15', 'sensor16', 'sensor17', 'sensor18',
         'sensor19', 'sensor20', 'sensor21']

train_data = pd.read_csv('/Users/johnpatanian/Documents/PHM_2016/PHM2016/train.txt', 
                         sep='\s+', header=None, names=tag_names)

# Create a Spark DataFrame from a pandas DataFrame
spark_train_data = sqlContext.createDataFrame(train_data)

## Step 4: Replicate some basic exploratory operations on the data set

### Example: Show the top 5 rows of data 

In [18]:
spark_train_data[['sensor1', 'sensor2', 'sensor3']].show(5)

+-------+-------+-------+
|sensor1|sensor2|sensor3|
+-------+-------+-------+
| 489.05| 604.13|1499.45|
| 518.67| 642.13|1584.55|
| 449.44| 555.42|1368.17|
| 491.19| 607.03|1488.44|
|  445.0| 549.52|1354.48|
+-------+-------+-------+
only showing top 5 rows



### Example: Descriptive statistics of a selected subset of columns

In [19]:
spark_train_data[['setting1', 'setting2', 'setting3']].describe().show()

+-------+------------------+------------------+------------------+
|summary|          setting1|          setting2|          setting3|
+-------+------------------+------------------+------------------+
|  count|             45918|             45918|             45918|
|   mean|24.008880656823496|0.5712513828999879|49.172437823947035|
| stddev|14.793676145732206|0.3109350455739768| 32.50365903446125|
|    min|               0.0|               0.0|               0.0|
|    max|            42.008|             0.842|             100.0|
+-------+------------------+------------------+------------------+



### Example: Get a list of data column names for the Sensor Only (Not the Settings)

In [20]:
def get_sensor_names(tag_names):
    """ Get tagnames starting with sensor.
    
    :param tag_names: Input time series data frame
    
    :return list of string tag names starting with sensor.
    """
    
    return [tag_name for tag_name in tag_names if re.search('^sensor.',tag_name)]

# Note I am using the Spark DataFrame's column property here.
sensor_columns = get_sensor_names(spark_train_data.columns)

# Now look at the top 5 rows of the first 8 columns the sensor data
spark_train_data[sensor_columns[0:8]].show(5)

+-------+-------+-------+-------+-------+-------+-------+-------+
|sensor1|sensor2|sensor3|sensor4|sensor5|sensor6|sensor7|sensor8|
+-------+-------+-------+-------+-------+-------+-------+-------+
| 489.05| 604.13|1499.45|1309.95|  10.52|  15.49| 394.88|2318.87|
| 518.67| 642.13|1584.55|1403.96|  14.62|  21.61| 553.67|2388.01|
| 449.44| 555.42|1368.17|1122.49|   5.48|    8.0| 194.93|2222.86|
| 491.19| 607.03|1488.44|1249.18|   9.35|  13.65| 334.82|2323.85|
|  445.0| 549.52|1354.48|1124.32|   3.91|   5.71| 138.24| 2211.8|
+-------+-------+-------+-------+-------+-------+-------+-------+
only showing top 5 rows



### Convert the DataFrame into a form that can be used in the MLLib library

In [21]:
import numpy as np

def convert_df_to_rdd(input_df):
    """ Convert to an rdd and then convert each row to a numpy array. """
    return input_df.rdd.map(lambda row: np.array(row))
    
settings_columns = ['setting1', 'setting2', 'setting3']
spark_train_rdd = convert_df_to_rdd(spark_train_data[sensor_columns])

spark_train_settings_rdd = convert_df_to_rdd(spark_train_data[settings_columns])

### Example: Do a k-means clustering of the settings data

In [22]:
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

clusters = KMeans.train(spark_train_settings_rdd, 6, maxIterations=10000, 
                        seed=0, initializationSteps=20)

centers = clusters.centers
centers

[array([ 20.00294648,   0.70049716,   0.        ]),
 array([ 25.00301261,   0.62051641,  80.        ]),
 array([ 42.00304399,   0.84051042,  40.        ]),
 array([  1.51675295e-03,   4.97670406e-04,   1.00000000e+02]),
 array([ 10.00296269,   0.25050253,  20.        ]),
 array([ 35.00305335,   0.84048928,  60.        ])]

This part is quite tricky in terms of adding the predicted column back onto the original data frame. For this we use the map column on the original data frame. The use of `row[2:5]` is for passing only the settings columns into the clusters.predict function.

In [24]:
column_list = spark_train_data.columns
column_list.insert(len(column_list), 'overall_setting')

spark_train_data = spark_train_data.map(lambda row: row + 
                     Row(overall_setting=clusters.predict(np.array(row[2:5])))).toDF(column_list)

### Drop the Individual Settings Columns and Replace with the Existing Columns

In [25]:
spark_train_data = spark_train_data.drop('setting1')
spark_train_data = spark_train_data.drop('setting2')
spark_train_data = spark_train_data.drop('setting3')

In [26]:
spark_train_data.groupBy('overall_setting').count().show()

+---------------+-----+
|overall_setting|count|
+---------------+-----+
|              0| 6771|
|              1| 6881|
|              2|11571|
|              3| 6954|
|              4| 6882|
|              5| 6859|
+---------------+-----+



### Example: Find the sensor data columns that have zero variance, grouped by setting

It might be a good idea to scale the sensors by doing a transformation here such as a z-score transformation. For information on how to do that, look at the documentation in [Spark Feature Extraction](http://spark.apache.org/docs/latest/mllib-feature-extraction.html#standardscaler).  In addition, it would also be good to check for outliers as well, and perhaps remove any rows that have outlier values. Here is an example function that could be used as a simple outlier detection method, that operates on an RDD. Note, that you could do something more complicated here, but I will leave this as an example.

```
def removeOutliers(nums):
    """Remove the outliers"""
    stats = nums.stats()
    stddev = math.sqrt(stats.variance())
    return nums.filter(lambda x: math.fabs(x - stats.mean()) < 3 * stddev)
```

In [47]:
aggregates = {sensor: "variance" for sensor in sensor_columns}

sens_var = spark_train_data.groupby('overall_setting').agg(aggregates)


sens_var[['overall_setting', 'variance(sensor1)', 'variance(sensor2)', 
          'variance(sensor3)', 'variance(sensor4)','variance(sensor5)']].show()

+---------------+-----------------+-------------------+------------------+------------------+-----------------+
|overall_setting|variance(sensor1)|  variance(sensor2)| variance(sensor3)| variance(sensor4)|variance(sensor5)|
+---------------+-----------------+-------------------+------------------+------------------+-----------------+
|              0|              0.0|0.21927179097491528| 33.01495534313832|60.846457765107196|              0.0|
|              1|              0.0|0.12439814689844039| 27.52905886471133| 48.07277211378348|              0.0|
|              2|              0.0| 0.1927775914975935|  32.0874425666109|57.060699897902325|              0.0|
|              3|              0.0|0.23371124362429516| 34.73643867941968|  76.2981558019152|              0.0|
|              4|              0.0|0.22357951137071927| 33.78095573922832| 68.88127197907136|              0.0|
|              5|              0.0|0.19300738910771909|31.979507268152656| 54.44673185521965|           

In [50]:
sens_var[['overall_setting', 'variance(sensor6)', 'variance(sensor7)', 
          'variance(sensor8)', 'variance(sensor9)']].show()

+---------------+--------------------+-------------------+--------------------+------------------+
|overall_setting|   variance(sensor6)|  variance(sensor7)|   variance(sensor8)| variance(sensor9)|
+---------------+--------------------+-------------------+--------------------+------------------+
|              0|2.162069229556021...|0.35978833315336467| 0.01655500043515558| 316.4515500635386|
|              1|1.852666848044716...|0.20060043956140963| 0.06902715061357538| 200.6590818837809|
|              2|2.437610582352507...|0.19596321175782855| 0.09108133170554643|326.92375705551024|
|              3|1.333659778434290...| 0.7316868477493585|0.004649571377442967|386.50922840707466|
|              4|2.206329159205174E-5| 0.4247177817242818|0.008727459587106043| 311.7814526976019|
|              5|2.844995374264426E-6| 0.2256633894896919| 0.07994953918910125|296.84603284430034|
+---------------+--------------------+-------------------+--------------------+------------------+



In [51]:
sens_var[['overall_setting', 'variance(sensor10)', 'variance(sensor11)', 
          'variance(sensor12)', 'variance(sensor13)']].show()

+---------------+--------------------+--------------------+-------------------+--------------------+
|overall_setting|  variance(sensor10)|  variance(sensor11)| variance(sensor12)|  variance(sensor13)|
+---------------+--------------------+--------------------+-------------------+--------------------+
|              0|1.801779986636029...| 0.05555042586039515|  0.238942085407681| 0.01721923376841796|
|              1|1.007091485485760...| 0.04596393928512227|0.11654133852316281| 0.07698369243503864|
|              2|2.114075759541546...|0.054931614164907785|0.11743622617178902| 0.10602108960718366|
|              3|                 0.0| 0.06596131962247459| 0.5036598995262795|0.004730644810546294|
|              4|                 0.0| 0.06233786870677962| 0.2751377504110462| 0.00907615756100676|
|              5|2.995734052463931...| 0.05275086644871066| 0.1432987302627138| 0.09197339462528958|
+---------------+--------------------+--------------------+-------------------+------------

In [52]:
sens_var[['overall_setting', 'variance(sensor14)', 'variance(sensor15)', 
          'variance(sensor16)', 'variance(sensor17)']].show()

+---------------+------------------+--------------------+--------------------+------------------+
|overall_setting|variance(sensor14)|  variance(sensor15)|  variance(sensor16)|variance(sensor17)|
+---------------+------------------+--------------------+--------------------+------------------+
|              0|243.97112676345705|0.001447754627586...|1.778954342385106...| 2.056442160251226|
|              1|170.94165872377084|0.001924382707956331|                 0.0|1.7161021417249334|
|              2|276.92093294863764|0.001359864768277...|                 0.0| 1.986614600758493|
|              3|284.51418242412683|0.001320814450505...|                 0.0|2.2133170036327012|
|              4|244.89686697315184|0.001393785450383...|                 0.0| 2.139088230562646|
|              5|242.63280641684977|0.001453575456791...|                 0.0|1.9484520320171814|
+---------------+------------------+--------------------+--------------------+------------------+



In [53]:
sens_var[['overall_setting', 'variance(sensor18)', 'variance(sensor19)', 
          'variance(sensor20)', 'variance(sensor21)']].show()

+---------------+------------------+------------------+--------------------+--------------------+
|overall_setting|variance(sensor18)|variance(sensor19)|  variance(sensor20)|  variance(sensor21)|
+---------------+------------------+------------------+--------------------+--------------------+
|              0|               0.0|               0.0|0.017760974082056014|0.006404840533218921|
|              1|               0.0|               0.0|0.012443469792113715|0.004497165941841051|
|              2|               0.0|               0.0|0.011613278439445065|0.004064782253391041|
|              3|               0.0|               0.0|0.030125807185357796|0.010910294475930301|
|              4|               0.0|               0.0|0.020826757809654116|0.007442464755276472|
|              5|               0.0|               0.0|0.012663789493752527|0.004770710653691737|
+---------------+------------------+------------------+--------------------+--------------------+



In [54]:
### Drop the Columns with Zero or Near Zero Variance
drop_columns = ['sensor1','sensor5','sensor18','sensor19','sensor8',
                'sensor11','sensor13','sensor15','sensor20','sensor21']

for column in drop_columns:
    spark_train_data = spark_train_data.drop(column)


In [56]:
spark_train_data.show(5)

+----+---------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+---------------+
|unit|cycle_num|sensor2|sensor3|sensor4|sensor6|sensor7|sensor9|sensor10|sensor12|sensor14|sensor16|sensor17|overall_setting|
+----+---------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+---------------+
|   1|        1| 604.13|1499.45|1309.95|  15.49| 394.88| 8770.2|    1.26|  372.15| 8120.83|    0.03|     368|              4|
|   1|        2| 642.13|1584.55|1403.96|  21.61| 553.67|9045.76|     1.3|  521.81| 8132.87|    0.03|     391|              3|
|   1|        3| 555.42|1368.17|1122.49|    8.0| 194.93|8343.91|    1.02|  183.26| 8063.84|    0.02|     334|              5|
|   1|        4| 607.03|1488.44|1249.18|  13.65| 334.82|8721.53|    1.08|  314.84|  8052.3|    0.02|     364|              0|
|   1|        5| 549.52|1354.48|1124.32|   5.71| 138.24|8314.56|    1.02|  130.44| 8083.67|    0.02|     330|         

### Example: Calculate and print out a correlation matrix of the sensor data

In [58]:
sensor_columns = get_sensor_names(spark_train_data.columns)
spark_train_rdd = convert_df_to_rdd(spark_train_data[sensor_columns])

print Statistics.corr(spark_train_rdd, method="pearson")

[[ 1.          0.98228306  0.98147887  0.94432695  0.9689343   0.97866876
   0.90700106  0.96913913  0.62563287  0.81142104  0.98311387]
 [ 0.98228306  1.          0.98970266  0.88478736  0.9290871   0.99781608
   0.93010454  0.92955412  0.75307104  0.80925269  0.99868712]
 [ 0.98147887  0.98970266  1.          0.91970741  0.95678185  0.98753589
   0.96209154  0.95718285  0.71635096  0.86164664  0.99040267]
 [ 0.94432695  0.88478736  0.91970741  1.          0.99327557  0.87638443
   0.87905722  0.99310776  0.40854184  0.84334845  0.88721497]
 [ 0.9689343   0.9290871   0.95678185  0.99327557  1.          0.92277106
   0.91946789  0.99999269  0.50896195  0.86048413  0.93107412]
 [ 0.97866876  0.99781608  0.98753589  0.87638443  0.92277106  1.
   0.93023928  0.92325907  0.77198508  0.80445931  0.99786454]
 [ 0.90700106  0.93010454  0.96209154  0.87905722  0.91946789  0.93023928
   1.          0.91991355  0.72702394  0.91299628  0.93096673]
 [ 0.96913913  0.92955412  0.95718285  0.99310776

### Example: Calculate the Principal Components of columns data